In [15]:
orcid = '0000-0003-0805-7952' # Fill your orcid here

In [16]:
%pip install requests
import requests
import os # for crossref?

Note: you may need to restart the kernel to use updated packages.


We use the `/works` api to list all works related to the orcid. This gives a summary of all works, so citation information is not included. We collect the `put-code` of all works to retrieve the citation information later.

In [17]:
# Step 1: Get all works from ORCID
response = requests.get('https://pub.orcid.org/v3.0/{}/works'.format(orcid),
                        headers={"Accept": "application/orcid+json" })
record = response.json()

In [18]:
put_codes = []
for work in record['group']:
    put_code = work['work-summary'][0]['put-code']
    put_codes.append(put_code)
put_code = put_codes[0]

citations = [] # moved this here from the for put_code in put_codes loop

In [19]:
# Step 2: Define helper to fetch BibTeX from Crossref
def get_bibtex_from_doi(doi):
    url = f'https://api.crossref.org/works/{doi}/transform/application/x-bibtex'
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"❌ Failed to fetch citation for DOI: {doi}")
        return None

We use the `/<orcid>/work/<put-code>` endpoint to retrieve the citation information for each record.

In [20]:
# Step 3: Loop through each work and collect citation
for put_code in put_codes:
    response = requests.get('https://pub.orcid.org/v3.0/{}/work/{}'.format(orcid, put_code),
                            headers={"Accept": "application/orcid+json" })
    work = response.json()
    
    # Try ORCID citation first
    if work.get('citation') and work['citation'].get('citation-value'):
        citations.append(work['citation']['citation-value'])
        continue
    
    # If no citation, try Crossref via DOI
    for ext_id in work.get('external-ids', {}).get('external-id', []):
        if ext_id['external-id-type'].lower() == 'doi':
            doi = ext_id['external-id-value']
            bibtex = get_bibtex_from_doi(doi)
            if bibtex:
                citations.append(bibtex)
            break  # Only use the first DOI found

In [21]:
# Step 4: Write all citations to pubs.bib
with open('pubs.bib', 'w') as bibfile:
    for citation in citations:
        bibfile.write(citation)
        bibfile.write('\n')

print(f"✅ Successfully saved {len(citations)} citations to pubs.bib")

✅ Successfully saved 8 citations to pubs.bib
